In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import random
from skimage.io import imread
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
main_folder = Path('.')

### 1. Reading images and creating labels

In [ ]:
def read_image_and_create_label_from_filepath(filepath):
    img = imread(filepath)
    label = filepath.parts[0]
    return img, label

In [ ]:
images, labels = zip(*[read_image_and_create_label_from_filepath(filepath) for filepath in main_folder.rglob("*.png")])

In [ ]:
images, labels = shuffle(images, labels)
images = np.asarray(images)
labels = np.asarray(labels)
print('Dataset and image sizes: {}'.format(images.shape))

### 2. Changing created labels into numbers 

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(labels)

### 3. Data Normalization

In [ ]:
print(images[0].max())
print(images[0].min())
images = images / 255
print(images[0].max())
print(images[0].min())

In [ ]:
plt.hist(labels)
print(np.unique(labels, return_counts=True))

### 4. Creating training, valid, test sets

In [ ]:
training_x, x_test, training_y, y_test = train_test_split(images, labels, test_size = 0.1, random_state = 42) 

In [ ]:
print('Number of elements in training_x: {}'.format(len(training_x)))
print('Number of elements in training_y: {}'.format(len(training_y)))
print('Number of elements in x_test: {}'.format(len(x_test)))
print('Number of elements in y_test: {}'.format(len(y_test)))

Split training sets into smaller training and validation sets - test dataset will be used at the very end

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(training_x, training_y, test_size = 0.2, random_state = 42) 

In [ ]:
print('Number of elements in x_train: {}'.format(len(x_train)))
print('Number of elements in y_train: {}'.format(len(y_train)))
print('Number of elements in x_valid: {}'.format(len(x_valid)))
print('Number of elements in y_valid: {}'.format(len(y_valid)))

In [ ]:
labels_names = list(np.unique(y_valid))
labels_dict = dict(zip(labels_names, ['pap', 'rock', 'sciss']))

In [ ]:
def print_imgs(x_set, y_set, names):
    fig, ax = plt.subplots(4, 4, figsize=(9, 9), sharex='col', sharey='row')
    for x in range(4):
        for y in range(4):
            idx = random.randint(0, len(x_set)-1)
            ax[x,y].imshow(x_set[idx], cmap='gray')
            ax[x,y].set_title(names[int(y_set[idx])])

In [ ]:
print_imgs(x_train, y_train, labels_dict)

### 5. Create a model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from sklearn.utils import class_weight

In [ ]:
model = tf.keras.Sequential()
model.add(Conv2D(9, (5, 5), activation='relu', input_shape=(200, 300, 3), name='L1'))
model.add(MaxPooling2D((2, 2), name='P1'))
model.add(Conv2D(9, (5, 5), activation='relu', name='L2'))
model.add(MaxPooling2D((2, 2), name='P2'))
model.add(Conv2D(9, (5, 5), activation='relu', name='L3'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_valid, y_valid))

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend(loc='upper right')

In [ ]:
plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
model.evaluate(x_valid, y_valid)

### 6. Test set

Analysis of hyperparameters:
- 10 epoch, batch_size=16, learning_rate=0.1 - [1.097268682042556, 0.37055838]
 
- 10 epoch, batch_size=32, learning_rate=0.1 - [1.1198053821896135, 0.3401015]

- 10 epoch, batch_size=64, learning_rate=0.1 - [1.094128987712505, 0.37055838]

- 10 epoch, batch_size=64, learning_rate=0.001 - [0.1878916552375416, 0.9407783]

- 10 epoch, batch_size=64, learning_rate=0.01 - [1.0990896372222254, 0.2893401]

- 10 epoch, batch_size=64, learning_rate=1.0 - [1.0991522888645102, 0.3401015]

- 10 epoch, batch_size=32, learning_rate=0.001 - [0.17450435519141227, 0.964467]

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
predictions = model.predict_classes(x_test)

In [ ]:
plt.hist(predictions)
print(np.unique(predictions, return_counts=True))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
def print_imgs_with_predictions(x_set, y_set, preds, names):
    fig, ax = plt.subplots(4, 4, figsize=(8, 8), sharex='col', sharey='row')
    for x in range(4):
        for y in range(4):
            idx = random.randint(0, x_set.shape[0]-1)
            ax[x,y].imshow(x_set[idx], cmap='gray')
            ax[x,y].set_title('y:{}, pr:{}'.format(names[int(y_set[idx])], names[int(preds[idx])]))

In [ ]:
wrong_indexes = [i for i, prediction in enumerate(predictions) if prediction != y_test[i]]
print('Number of wrong predictions: {}'.format(len(wrong_indexes)))
print('Number of samples in x_test: {}'.format(len(x_test)))

#### Predictions:

In [ ]:
print_imgs_with_predictions(x_test, y_test, predictions, labels_dict)

#### Wrong predictions:

In [ ]:
print_imgs_with_predictions(x_test[wrong_indexes], y_test[wrong_indexes], predictions[wrong_indexes], labels_dict)